In [2]:
import requests
import pandas as pd
import os 
import numpy as np
from sqlalchemy import create_engine

In [3]:
# variable to hold path to file
path = "Resources/PoGoKaggle.csv"

In [10]:
# read file using path
kaggle = pd.read_csv(path)


In [11]:
# Create a total stats column
kaggle['Stats'] = kaggle['Defense'] + kaggle['Attack']

In [13]:
#Make Legendary nomenclature  Binary
kaggle = kaggle.replace(regex={'No': False, 'Sí' : True})


In [14]:
# Reorganize the table
Org_kaggle = kaggle[['Pokedex' , 'Name', 'Generation' , 'Primary' , 'Secondary' , 'MaxHP', 'MaxCP' , 'Stamina' , 'Attack' , 'Defense' , 'Stats' , 
       'Capture_rate' , 'Escape_rate', 'Weight' , 'Height' , 'Legendary']]

In [15]:
# Split the databse into 3 tables and present middle table
Split_kaggle = np.split(Org_kaggle, [5,11] , axis =1)
Split_kaggle[1]

,MaxHP,MaxCP,Stamina,Attack,Defense,Stats
0,113,1115,128,118,111,229
1,134,1699,155,151,143,294
2,162,2720,190,198,189,387
3,105,980,118,116,93,209
4,131,1653,151,158,126,284
...,...,...,...,...,...,...
644,173,3588,205,261,182,443
645,205,3575,245,246,170,416
646,177,3698,209,260,192,452
647,189,3972,225,250,225,475


In [16]:
# Create a new variable to hold the dataframe, set the index as a primary key, then drop dupliate rows
Battle_K = Split_kaggle[1]
Battle_K['Primary_Key'] = Battle_K.index
Battle_K = Battle_K.drop_duplicates( subset =  ['MaxHP', 'MaxCP' , 'Stamina' , 'Attack' , 'Defense' , 'Stats'])
Battle_K

,MaxHP,MaxCP,Stamina,Attack,Defense,Stats,Primary_Key
0,113,1115,128,118,111,229,0
1,134,1699,155,151,143,294,1
2,162,2720,190,198,189,387,2
3,105,980,118,116,93,209,3
4,131,1653,151,158,126,284,4
...,...,...,...,...,...,...,...
640,160,3345,188,266,164,430,640
644,173,3588,205,261,182,443,644
645,205,3575,245,246,170,416,645
647,189,3972,225,250,225,475,647


In [17]:
# Create placeholder for foreign key
Org_kaggle['Battle_FK'] = 0
# Make foreign key for reference for SQL
for index, row in Org_kaggle.iterrows():
    for index2, row2 in Battle_K.iterrows():
        #         If the battle rows match the row drop the Foriegn key in the foriegn key column
        if row[5] == row2[0] and row[6] == row2[1] and row[7] == row2[2] and row[8] == row2[3] and row[9] == row2[4] and row[10] == row2[5]:
            Org_kaggle.loc[index, 'Battle_FK'] = int(row2[6])
            

C:\Users\Lenny Ovo\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Lenny Ovo\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [18]:
# Create a new variable to hold the dataframe, set the index as a primary key, then drop dupliate rows
Catch_K= Split_kaggle[2]
Catch_K['C_Primary_Key'] = Catch_K.index
Catch_K = Catch_K.drop_duplicates( subset =  ['Capture_rate' , 'Escape_rate', 'Weight' , 'Height' , 'Legendary'])
Catch_K

,Capture_rate,Escape_rate,Weight,Height,Legendary,C_Primary_Key
0,0.2,0.1,6.9,0.7,False,0
1,0.1,0.07,13,1,False,1
2,0.05,0.05,100,2,False,2
3,0.2,0.1,8.5,0.6,False,3
4,0.1,0.07,19,1.1,False,4
...,...,...,...,...,...,...
644,0.02,0.01,68,1.5,True,644
645,0.02,0.01,325,3,True,645
646,0.02,0.01,48.5,1.4,False,646
647,0.02,0.01,6.5,0.6,False,647


In [19]:
# Create placeholder for foreign key
Org_kaggle['Catch_FK'] = 0
# Make foreign key for reference for SQL
for index, row in Org_kaggle.iterrows():
    for index2, row2 in Catch_K.iterrows():
        #         If the catch rows match the row drop the Foriegn key in the foriegn key column
        if row[11] == row2[0] and row[12] == row2[1] and row[13] == row2[2] and row[14] == row2[3] and row[15] == row2[4]:
            Org_kaggle.loc[index, 'Catch_FK'] = int(row2[5])
            

C:\Users\Lenny Ovo\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Lenny Ovo\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [20]:
# Select columns wanted for name dataframe, uncapitalize columns then set the index as the primary key
name_k = Org_kaggle[['Pokedex' , 'Name', 'Generation' , 'Primary' , 'Secondary','Catch_FK' , 'Battle_FK']]
name_k = name_k.rename(columns = {'Pokedex' : 'pokedex', 'Name' :'name', 'Generation' :'generation' , 'Primary' :'primary_type' , 'Secondary' : 'secondary_type','Catch_FK'  :'catch_fk' , 'Battle_FK' : 'battle_fk'})
name_k['pk'] = name_k.index
name_k

,pokedex,name,generation,primary_type,secondary_type,catch_fk,battle_fk,pk
0,1,Bulbasaur,1,Grass,Poison,0,0,0
1,2,Ivysaur,1,Grass,Poison,1,1,1
2,3,Venusaur,1,Grass,Poison,2,2,2
3,4,Charmander,1,Fire,False,3,3,3
4,5,Charmeleon,1,Fire,False,4,4,4
...,...,...,...,...,...,...,...,...
644,645,Landorus,5,Ground,Flying,644,644,644
645,646,Kyurem,5,Dragon,Ice,645,645,645
646,647,Keldeo,5,Water,Fighting,646,638,646
647,648,Meloetta,5,False,Psychic,647,647,647


In [21]:
# uncapitalize columns
Catch_K = Catch_K.rename(columns = {'Capture_rate' : 'capture_rate' , 'Escape_rate' :'escape_rate', 'Weight' : 'weight' , 'Height' : 'height' , 'Legendary':'legendary' , 'C_Primary_Key' : 'c_primary_key'})
Catch_K




,capture_rate,escape_rate,weight,height,legendary,c_primary_key
0,0.2,0.1,6.9,0.7,False,0
1,0.1,0.07,13,1,False,1
2,0.05,0.05,100,2,False,2
3,0.2,0.1,8.5,0.6,False,3
4,0.1,0.07,19,1.1,False,4
...,...,...,...,...,...,...
644,0.02,0.01,68,1.5,True,644
645,0.02,0.01,325,3,True,645
646,0.02,0.01,48.5,1.4,False,646
647,0.02,0.01,6.5,0.6,False,647


In [22]:
# uncapitalize columns

Battle_K = Battle_K.rename(columns = {'MaxHP' :  'maxhp' , "MaxCP" : 'maxcp' , "Stamina" : 'stamina' , 'Attack' : 'attack' ,  'Defense' : 'defense' , 'Stats' : 'stats' , 'Primary_Key' : 'primary_key'})
Battle_K

,maxhp,maxcp,stamina,attack,defense,stats,primary_key
0,113,1115,128,118,111,229,0
1,134,1699,155,151,143,294,1
2,162,2720,190,198,189,387,2
3,105,980,118,116,93,209,3
4,131,1653,151,158,126,284,4
...,...,...,...,...,...,...,...
640,160,3345,188,266,164,430,640
644,173,3588,205,261,182,443,644
645,205,3575,245,246,170,416,645
647,189,3972,225,250,225,475,647


In [23]:
# Create Connection with ash user and create engine
rds_connection_string = "ash:catchem@localhost:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [25]:
# Check tables
engine.table_names()

['rapid_catch',
 'rapid_name',
 'kaggle_battle',
 'rapid_battle',
 'kaggle_catch',
 'kaggle_name']

In [26]:
# push Catch data to catch table on DB for kaggle tables
Catch_K.to_sql(name='kaggle_catch', con=engine, if_exists='append', index=False)

In [27]:
# push battle data to battle table on DB for kaggle tables
Battle_K.to_sql(name='kaggle_battle', con=engine, if_exists='append', index=False)

In [29]:
# push name data to name table on DB for kaggle tables
name_k.to_sql(name='kaggle_name', con=engine, if_exists='append', index=False)

In [30]:
# Check that push was successful
pd.read_sql_query('select * from kaggle_battle', con=engine).head()

,maxhp,maxcp,stamina,attack,defense,stats,primary_key
0,113,1115,128,118,111,229,0
1,134,1699,155,151,143,294,1
2,162,2720,190,198,189,387,2
3,105,980,118,116,93,209,3
4,131,1653,151,158,126,284,4


In [31]:
# Check that push was successful
pd.read_sql_query('select * from kaggle_catch', con=engine).head()

,capture_rate,escape_rate,weight,height,legendary,c_primary_key
0,0.20,0.10,6.9,0.7,False,0
1,0.10,0.07,13.0,1.0,False,1
2,0.05,0.05,100.0,2.0,False,2
3,0.20,0.10,8.5,0.6,False,3
4,0.10,0.07,19.0,1.1,False,4


In [32]:
# Check that push was successful
pd.read_sql_query('select * from kaggle_name', con=engine).head()

,pokedex,name,generation,primary_type,secondary_type,catch_fk,battle_fk,pk
0,1,Bulbasaur,1,Grass,Poison,0,0,0
1,2,Ivysaur,1,Grass,Poison,1,1,1
2,3,Venusaur,1,Grass,Poison,2,2,2
3,4,Charmander,1,Fire,false,3,3,3
4,5,Charmeleon,1,Fire,false,4,4,4
